# **Rebalancing Tool - AV** 
    
This tool is used to validate if funds on AV platform are within predefined tolerance ranges.

In [1]:
import pandas as pd

from utils.glidepath import merged_glidepaths
import utils.pipeline as pl
import utils.validations as val
import utils.config as config


## Settings


In [2]:
PROVIDER = 'AV'

# File paths
av_weekly = 'data/av_weekly'
av_reference = pd.read_csv('data/reference/av_reference.csv')
av_static_funds_targets = pd.read_csv('data/reference/av_static_funds_targets.csv')
av_output = 'data/output/av_output.csv'

# Tolerance levels
MIN_RANGE = -0.03
MAX_RANGE = 0.03

glidepath_lookback = 'YES'

webhook_url = config.webhook_url


## Processing

### Preprocess data

In [3]:

all_glidepaths = merged_glidepaths()
df = pl.load_and_preprocess_data_av(av_weekly, av_reference)
print(df.info())    



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             720 non-null    datetime64[ns]
 1   fund_label       720 non-null    object        
 2   fund_key         720 non-null    object        
 3   valuation        720 non-null    float64       
 4   actual_weight    720 non-null    float64       
 5   target_weight    720 non-null    float64       
 6   fund_glidepath   278 non-null    object        
 7   fund_underlying  472 non-null    object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 45.1+ KB
None


### Test

In [4]:
val.validate_columns(df, pl.pre_columns)
val.test_no_duplicates(df)  


Validation Result: Column date has been validated
Validation Result: Column fund_label has been validated
Validation Result: Column fund_underlying has been validated
Validation Result: Column fund_key has been validated
Validation Result: Column fund_glidepath has been validated
Validation Result: Column valuation has been validated
Validation Result: Column actual_weight has been validated
Validation Result: Column target_weight has been validated
Validation Result: No duplicates found



### Main processing and calculations

In [5]:
df = pl.add_glidepath_data(df, glidepath_lookback)
df = pl.add_lookup_values(df, all_glidepaths)
df = pl.add_static_target_values(df, av_static_funds_targets)
df = pl.calculate_difference_final(df)

df.to_csv(av_output, index=False)
print('Data processed and saved to data/output/av_output.csv')
print('')




Lookup values added successfully and count test is passed.

Validation test: No negative values found in the column "target_val"

Data processed and saved to data/output/av_output.csv



### Adding mask providing final output

In [6]:
validated_df, date_result = pl.check_range(df, MIN_RANGE, MAX_RANGE)
validated_df = pl.post_transformations(validated_df)
print(date_result)


2024-05-31


### Test

In [7]:
val.test_no_invalid_dates(validated_df)



Validation Result: Dates are in range 


## Send Message

In [8]:
#pl.send_dataframe_to_teams(validated_df, webhook_url, provider=PROVIDER)
